<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/images/IDSN-logo.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Getting Started with PySpark and Pandas


Estimated time needed: **60** minutes


PySpark is the Python API for Apache Spark, a distributed computing system designed for handling large-scale data processing. Its ability to perform computations across multiple nodes makes it an ideal choice for Big Data scenarios, where data sets can reach terabytes or even petabytes in size. PySpark allows data scientists and analysts to harness the power of distributed computing, scaling up their workflows and processing capabilities significantly.


In contrast, Pandas is a powerful library tailored for data manipulation and analysis in Python, primarily used for handling structured data. It provides rich functionality for data cleaning, transformation, aggregation, and visualization, making it particularly suited for smaller data sets typically fitting into memory. Pandas excels at operations requiring quick turnaround times for data analysis and exploration, facilitating tasks such as data wrangling and exploratory data analysis.


To illustrate the strengths of both PySpark and Pandas, we will utilize a sample COVID-19 data set containing daily cases, deaths, and vaccinations across various continents.


## Objectives


- Understand PySpark and Pandas: Explain the core functionalities and use cases of PySpark for big data processing and Pandas for data manipulation.


- Set up the environment: Install and configure PySpark and Pandas to work together in a Python environment.


- Load and explore data: Import data into Pandas and PySpark DataFrames and perform basic data exploration.


- Convert between DataFrames: Convert a Pandas DataFrame to a Spark DataFrame for distributed processing.


- Perform data manipulation: Create new columns, filter data, and perform aggregations using PySpark.


- Utilize SQL queries: Use Spark SQL for querying data and leveraging user-defined functions (UDFs).


## 1. PySpark overview


### Overview


PySpark is the Python API for Apache Spark, designed for large-scale data processing and analysis. It offers tools for working with RDDs and DataFrames, enabling efficient, fault-tolerant distributed computing.


### Key features


- **Distributed computing:** Handles data across multiple nodes in a cluster.
- **High performance:** Outperforms traditional frameworks in speed.
- **Big data handling:** Manages data sets larger than a single machine's memory.
- **Python integration:** Compatible with Python libraries like Pandas and NumPy.


### Use cases


- **Large-scale data processing:** Ideal for processing large volumes of data that exceed the capacity of a single machine.
- **Data analysis:** Useful for complex data manipulations and analysis using distributed computing.


### Strengths


- High-speed data processing.
- Fault-tolerant and scalable.


### Limitations


- Complex setup and configuration.
- Steeper learning curve compared to some data processing tools.


## 2. Understanding Pandas


### Overview


Pandas is a Python library designed for data manipulation and analysis. It provides two primary data structures: Series and DataFrame, which facilitate handling and organizing structured data.


### Key features


- **Data structures:** Series for one-dimensional data and DataFrame for two-dimensional data.
- **Data I/O:** Reads and writes data in various formats such as CSV, Excel, and SQL.
- **Data cleaning:** Functions for handling missing or duplicate data.
- **Data analysis:** Includes statistical functions for detailed data analysis.


### Use cases


- **Data manipulation:** Efficient handling of structured data.
- **Data analysis:** Comprehensive analysis and transformation of data sets.


### Strengths


- User-friendly API for data manipulation.
- Extensive support for various data formats.


### Limitations


- Limited scalability for extremely large datasets compared to distributed frameworks.


## 3. Setting up the environment


### Installation


- First, let's install the necessary libraries if they are not already installed.


In [ ]:
!pip install pyspark

In [ ]:
!pip install findspark

In [3]:
!pip install --force-reinstall "numpy==1.23.5" "pandas==1.5.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 3.9 MB/s eta 0:00:0000:0100:01
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-dateutil-2.9.0.post0:
      Successfully uninstalled python-dateu

In [ ]:
!pip install pyarrow

## 4. Initializing a Spark session


A Spark session is crucial for working with PySpark. It enables DataFrame creation, data loading, and various operations.


### Importing libraries


- `findspark` is used to locate the Spark installation.
- `pandas` is imported for data manipulation.


### Creating a Spark session


- `SparkSession.builder.appName("COVID-19 Data Analysis").getOrCreate()` initializes a Spark session with the specified application name.


### Checking Spark session


- The code checks if the Spark session is active and prints an appropriate message.


In [1]:
import findspark  # This helps us find and use Apache Spark
findspark.init()  # Initialize findspark to locate Spark
from pyspark.sql import SparkSession  
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DateType
import pandas as pd  
# Initialize a Spark Session
spark = SparkSession \
    .builder \
    .appName("COVID-19 Data Analysis") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

# Check if the Spark Session is active
if 'spark' in locals() and isinstance(spark, SparkSession):
    print("SparkSession is active and ready to use.")
else:
    print("SparkSession is not active. Please create a SparkSession.")

your 131072x1 screen size is bogus. expect trouble


25/12/02 15:55:12 WARN Utils: Your hostname, DiegoSrz resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/12/02 15:55:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/12/02 15:55:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
SparkSession is active and ready to use.


## 5. Importing data into Pandas from various sources


Let's read the COVID-19 data from the provided URL.


In [2]:
# Read the COVID-19 data from the provided URL
vaccination_data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/KpHDlIzdtR63BdTofl1mOg/owid-covid-latest.csv')
vaccination_data.head(3)

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2024-08-04,235214.0,0.0,0.000,7998.0,0.0,0.0,...,NaN,37.746,0.50,64.83,0.511,4.112877e+07,NaN,NaN,NaN,NaN
1,OWID_AFR,NaN,Africa,2024-08-04,13145380.0,36.0,5.143,259117.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN,NaN
2,ALB,Europe,Albania,2024-08-04,335047.0,0.0,0.000,3605.0,0.0,0.0,...,51.2,NaN,2.89,78.57,0.795,2.842318e+06,NaN,NaN,NaN,NaN


## 6. Displaying the first five records


### To retrieve and print the first five records


- `vaccination_data.head()` retrieves the first five rows of the DataFrame vaccination_data.This gives us a quick look at the data contained within the data set.
- The `print` function is used to display a message indicating what is being shown, followed by the actual data.


### Selecting specific columns:


- Let\'s define a list called `columns_to_display`, which contains the names of the columns as : `['continent', 'total_cases', 'total_deaths', 'total_vaccinations', 'population']`.
- By using `vaccination_data[columns_to_display].head()`, let\'s filter the DataFrame to only show the specified columns and again display the first five records of this subset.
- The continent column is explicitly converted to string, while the numeric columns (total cases, total deaths, total vaccinations, population) are filled with zeros for NaN values and then converted to int64 (which is compatible with LongType in Spark).
- The use of fillna(0) ensures that NaN values do not cause type issues during the Spark DataFrame creation.


In [3]:
print("Displaying the first 5 records of the vaccination data:")
columns_to_display = ['continent', 'total_cases', 'total_deaths', 'total_vaccinations', 'population']
# Show the first 5 records
print(vaccination_data[columns_to_display].head())

Displaying the first 5 records of the vaccination data:
  continent  total_cases  total_deaths  total_vaccinations    population
0      Asia     235214.0        7998.0                 NaN  4.112877e+07
1       NaN   13145380.0      259117.0                 NaN  1.426737e+09
2    Europe     335047.0        3605.0                 NaN  2.842318e+06
3    Africa     272139.0        6881.0                 NaN  4.490323e+07
4   Oceania       8359.0          34.0                 NaN  4.429500e+04


## 7. Converting the Pandas DataFrame to a Spark DataFrame


Let\'s convert the Pandas DataFrame, which contains our COVID-19 vaccination data, into a Spark DataFrame. This conversion is crucial as it allows us to utilize Spark\'s distributed computing capabilities, enabling us to handle larger datasets and perform operations in a more efficient manner.


### Defining the schema:


- **StructType**: 
  - A class that defines a structure for a DataFrame.

- **StructField**: 
  - Represents a single field in the schema.
  - **Parameters**:
    1. **Field name**: The name of the field.
    2. **Data type**: The type of data for the field.
    3. **Nullable**: A boolean indicating whether null values are allowed.

- **Data types**:
  - **StringType()**: Used for text fields.
  - **LongType()**: Used for numerical fields.


### Data type conversion:


- **astype(str)**: 
  - Used to convert the `'continent'` column to string type.

- **fillna(0)**: 
  - Replaces any NaN values with 0, ensuring that the numerical fields do not contain any missing data.

- **astype('int64')**: 
  - Converts the columns from potentially mixed types to 64-bit integers for consistent numerical representation.


### Creating a Spark DataFrame:


- **createDataFrame**:
  - The `createDataFrame` method of the Spark session (`spark`) is called with `vaccination_data` (the Pandas DataFrame) as its argument.
  - **Parameters**:
    - It takes as input a subset of the pandas DataFrame that corresponds to the fields defined in the schema, accessed using `schema.fieldNames()`.
- This function automatically converts the Pandas DataFrame into a Spark DataFrame, which is designed to handle larger data sets across a distributed environment.


- The resulting spark_df will have the defined schema, which ensures consistency and compatibility with Spark's data processing capabilities.


### Storing the result:


In [4]:
# Convert to Spark DataFrame directly
# Define the schema
schema = StructType([
    StructField("continent", StringType(), True),
    StructField("total_cases", LongType(), True),
    StructField("total_deaths", LongType(), True),
    StructField("total_vaccinations", LongType(), True),
    StructField("population", LongType(), True)
])

# Convert the columns to the appropriate data types
vaccination_data['continent'] = vaccination_data['continent'].astype(str)  # Ensures continent is a string
vaccination_data['total_cases'] = vaccination_data['total_cases'].fillna(0).astype('int64')  # Fill NaNs and convert to int
vaccination_data['total_deaths'] = vaccination_data['total_deaths'].fillna(0).astype('int64')  # Fill NaNs and convert to int
vaccination_data['total_vaccinations'] = vaccination_data['total_vaccinations'].fillna(0).astype('int64')  # Fill NaNs and convert to int
vaccination_data['population'] = vaccination_data['population'].fillna(0).astype('int64')  # Fill NaNs and convert to int

spark_df = spark.createDataFrame(vaccination_data[schema.fieldNames()])  # Use only the specified fields
# Show the Spark DataFrame
spark_df.show()

/opt/spark/python/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


+-------------+-----------+------------+------------------+----------+
|    continent|total_cases|total_deaths|total_vaccinations|population|
+-------------+-----------+------------+------------------+----------+
|         Asia|     235214|        7998|                 0|  41128772|
|          nan|   13145380|      259117|                 0|1426736614|
|       Europe|     335047|        3605|                 0|   2842318|
|       Africa|     272139|        6881|                 0|  44903228|
|      Oceania|       8359|          34|                 0|     44295|
|       Europe|      48015|         159|                 0|     79843|
|       Africa|     107481|        1937|                 0|  35588996|
|North America|       3904|          12|                 0|     15877|
|North America|       9106|         146|                 0|     93772|
|South America|   10101218|      130663|                 0|  45510324|
|         Asia|     452273|        8777|                 0|   2780472|
|North

## 8. Checking the structure of the Spark DataFrame


In this section, Let\'s examine the structure of the Spark DataFrame that we created from the Pandas DataFrame. Understanding the schema of a DataFrame is crucial as it provides insight into the data types of each column and helps ensure that the data is organized correctly for analysis.


### Displaying the schema:

- The method `spark_df.printSchema()` is called to output the structure of the Spark DataFrame.
- This method prints the names of the columns along with their data types (e.g., `StringType`, `IntegerType`, `DoubleType`, etc.), providing a clear view of how the data is organized.


In [5]:
print("Schema of the Spark DataFrame:")
spark_df.printSchema()
# Print the structure of the DataFrame (columns and types)

Schema of the Spark DataFrame:
root
 |-- continent: string (nullable = true)
 |-- total_cases: long (nullable = true)
 |-- total_deaths: long (nullable = true)
 |-- total_vaccinations: long (nullable = true)
 |-- population: long (nullable = true)



## 9. Basic data exploration


In this section, let\'s perform basic data exploration on the Spark DataFrame. This step is essential for understanding the data set better, allowing us to gain insights and identify any patterns or anomalies. Let\'s demonstrate how to view specific contents of the DataFrame, select certain columns, and filter records based on conditions.


### 9.1 Viewing DataFrame contents


- To view the contents in the DataFrame, use the following code:


In [6]:
# List the names of the columns you want to display
columns_to_display = ['continent', 'total_cases', 'total_deaths', 'total_vaccinations', 'population']
# Display the first 5 records of the specified columns
spark_df.select(columns_to_display).show(5)

+---------+-----------+------------+------------------+----------+
|continent|total_cases|total_deaths|total_vaccinations|population|
+---------+-----------+------------+------------------+----------+
|     Asia|     235214|        7998|                 0|  41128772|
|      nan|   13145380|      259117|                 0|1426736614|
|   Europe|     335047|        3605|                 0|   2842318|
|   Africa|     272139|        6881|                 0|  44903228|
|  Oceania|       8359|          34|                 0|     44295|
+---------+-----------+------------+------------------+----------+
only showing top 5 rows



### 9.2 Picking specific columns


- To display only certain columns, use the following code:


In [7]:
print("Displaying the 'continent' and 'total_cases' columns:")
# Show only the 'continent' and 'total_cases' columns
spark_df.select('continent', 'total_cases').show(5)

Displaying the 'continent' and 'total_cases' columns:
+---------+-----------+
|continent|total_cases|
+---------+-----------+
|     Asia|     235214|
|      nan|   13145380|
|   Europe|     335047|
|   Africa|     272139|
|  Oceania|       8359|
+---------+-----------+
only showing top 5 rows



### 9.3 Sifting Through Data


- To filter records based on a specific condition, use the following code:


In [12]:
from pyspark.sql.functions import desc

print("Filtering records where 'total_cases' is greater than 1,000,000:")
 # Show records with more than 1 million total cases
spark_df.\
filter(spark_df['total_cases'] > 1000000).\
orderBy(desc('total_cases')).\
show(5) 

Filtering records where 'total_cases' is greater than 1,000,000:
+---------+-----------+------------+------------------+----------+
|continent|total_cases|total_deaths|total_vaccinations|population|
+---------+-----------+------------+------------------+----------+
|      nan|  775866783|     7057132|       13578774356|7975105024|
|      nan|  429044049|     2997359|        2840880020|1250514600|
|      nan|  301499099|     1637249|        9104304615|4721383370|
|      nan|  252916868|     2102483|        1399334208| 744807803|
|      nan|  251753518|     2824452|        5449980941|2525921300|
+---------+-----------+------------+------------------+----------+
only showing top 5 rows



## 10. Working with columns


In this section, let\'s create a new column called `death_percentage`, which calculates the death rate during the COVID-19 pandemic. This calculation is based on the total_deaths (the count of deaths) and the population (the total population) columns in our Spark DataFrame. This new metric will provide valuable insight into the impact of COVID-19 in different regions.


- Let\'s import the functions module from `pyspark.sql` as `F`, which contains built-in functions for DataFrame operations.


### Calculating the death percentage:


- Let\'s create a new DataFrame `spark_df_with_percentage` by using the `withColumn()` method to add a new column called `death_percentage`.
- The formula `(spark_df['total_deaths'] / spark_df['population']) * 100` computes the death percentage by dividing the total deaths by the total population and multiplying by 100.


In [13]:
from pyspark.sql import functions as F

spark_df_with_percentage = spark_df.withColumn(
    'death_percentage', 
    (spark_df['total_deaths'] / spark_df['population']) * 100
)

spark_df_with_percentage.show(5)

+---------+-----------+------------+------------------+----------+--------------------+
|continent|total_cases|total_deaths|total_vaccinations|population|    death_percentage|
+---------+-----------+------------+------------------+----------+--------------------+
|     Asia|     235214|        7998|                 0|  41128772| 0.01944624069981958|
|      nan|   13145380|      259117|                 0|1426736614|0.018161516110078605|
|   Europe|     335047|        3605|                 0|   2842318|  0.1268330989002638|
|   Africa|     272139|        6881|                 0|  44903228|0.015324065343364624|
|  Oceania|       8359|          34|                 0|     44295| 0.07675809910825149|
+---------+-----------+------------+------------------+----------+--------------------+
only showing top 5 rows



### Formatting the percentage:


- Let\'s update the `death_percentage` column to format its values to two decimal places using `F.format_number()`, and concatenate a percentage symbol using `F.concat()` and `F.lit('%')`.
- This makes the death percentage easier to read and interpret.


In [14]:

spark_df_with_percentage = spark_df_with_percentage.withColumn(
    'death_percentage',
    F.concat(
        # Format to 2 decimal places
        F.format_number(spark_df_with_percentage['death_percentage'], 2), 
        # Append the percentage symbol 
        F.lit('%')  
    )
)
spark_df_with_percentage.show(5)

+---------+-----------+------------+------------------+----------+----------------+
|continent|total_cases|total_deaths|total_vaccinations|population|death_percentage|
+---------+-----------+------------+------------------+----------+----------------+
|     Asia|     235214|        7998|                 0|  41128772|           0.02%|
|      nan|   13145380|      259117|                 0|1426736614|           0.02%|
|   Europe|     335047|        3605|                 0|   2842318|           0.13%|
|   Africa|     272139|        6881|                 0|  44903228|           0.02%|
|  Oceania|       8359|          34|                 0|     44295|           0.08%|
+---------+-----------+------------+------------------+----------+----------------+
only showing top 5 rows



### Selecting relevant columns:


- Let\'s define a list `columns_to_display` that includes `'total_deaths', 'population', 'death_percentage', 'continent', 'total_vaccinations', and 'total_cases'`.
- Finally, let's display the first five records of the modified DataFrame with the new column by calling `spark_df_with_percentage.select(columns_to_display).show(5)`.


In [15]:
columns_to_display = ['total_deaths', 'population', 'death_percentage', 'continent', 'total_vaccinations', 'total_cases']
spark_df_with_percentage.select(columns_to_display).show(5)

+------------+----------+----------------+---------+------------------+-----------+
|total_deaths|population|death_percentage|continent|total_vaccinations|total_cases|
+------------+----------+----------------+---------+------------------+-----------+
|        7998|  41128772|           0.02%|     Asia|                 0|     235214|
|      259117|1426736614|           0.02%|      nan|                 0|   13145380|
|        3605|   2842318|           0.13%|   Europe|                 0|     335047|
|        6881|  44903228|           0.02%|   Africa|                 0|     272139|
|          34|     44295|           0.08%|  Oceania|                 0|       8359|
+------------+----------+----------------+---------+------------------+-----------+
only showing top 5 rows



## 11. Grouping and summarizing


 Let\'s calculate the total number of deaths per continent using the data in our Spark DataFrame. Grouping and summarizing data is a crucial aspect of data analysis, as it allows us to aggregate information and identify trends across different categories.


 ### Grouping the data


The `spark_df.groupby(['continent'])` method groups the data by the `continent` column. This means that all records associated with each continent will be aggregated together.


### Aggregating the deaths


The `agg({"total_deaths": "SUM"})` function is used to specify the aggregation operation. In this case, we want to calculate the sum of the `total_deaths` for each continent. This operation will create a new DataFrame where each continent is listed alongside the total number of deaths attributed to it.


### Displaying the results


The `show()` method is called to display the results of the aggregation. This will output the total number of deaths for each continent in a tabular format.


In [16]:
print("Calculating the total deaths per continent:")
# Group by continent and sum total death rates
spark_df.groupby(['continent']).agg({"total_deaths": "SUM"}).show()  

Calculating the total deaths per continent:


+-------------+-----------------+
|    continent|sum(total_deaths)|
+-------------+-----------------+
|       Europe|          2102483|
|       Africa|           259117|
|          nan|         22430618|
|North America|          1671178|
|South America|          1354187|
|      Oceania|            32918|
|         Asia|          1637249|
+-------------+-----------------+



## 12. Exploring user-defined functions (UDFs)



UDFs in PySpark allow us to create custom functions that can be applied to individual columns within a DataFrame. This feature provides increased flexibility and customization in data processing, enabling us to define specific transformations or calculations that are not available through built-in functions. In this section, let\'s define a UDF to convert total deaths in the dataset.


### Importing pandas_udf


The `pandas_udf` function is imported from `pyspark.sql.functions`. This decorator allows us to define a UDF that operates on Pandas Series


### Defining the UDF


This function `convert_total_deaths()` takes in a parameter `total_deaths` and returns double its value. You can replace the logic with any transformation you want to apply to the column data.



### Registering the UDF


The line `spark.udf.register("convert_total_deaths", convert_total_deaths, IntegerType())` registers the UDF with Spark indicating that the function returns an integer, allowing us to use it in Spark SQL queries and DataFrame operations.


In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
# Function definition
def convert_total_deaths(total_deaths):
    return total_deaths * 2
# Here you can define any transformation you want
# Register the UDF with Spark
spark.udf.register("convert_total_deaths", convert_total_deaths, IntegerType())

<function __main__.convert_total_deaths(total_deaths)>

## 13. Using Spark SQL


Spark SQL enables us to execute SQL queries directly on DataFrames.


In [18]:
# Drop the existing temporary view if it exists
spark.sql("DROP VIEW IF EXISTS data_v")

# Create a new temporary view
spark_df.createTempView('data_v')

# Execute the SQL query using the UDF
spark.sql('SELECT continent, total_deaths, convert_total_deaths(total_deaths) as converted_total_deaths FROM data_v').show()

+-------------+------------+----------------------+
|    continent|total_deaths|converted_total_deaths|
+-------------+------------+----------------------+
|         Asia|        7998|                 15996|
|          nan|      259117|                518234|
|       Europe|        3605|                  7210|
|       Africa|        6881|                 13762|
|      Oceania|          34|                    68|
|       Europe|         159|                   318|
|       Africa|        1937|                  3874|
|North America|          12|                    24|
|North America|         146|                   292|
|South America|      130663|                261326|
|         Asia|        8777|                 17554|
|North America|         292|                   584|
|          nan|     1637249|               3274498|
|      Oceania|       25236|                 50472|
|       Europe|       22534|                 45068|
|         Asia|       10353|                 20706|
|North Ameri

## 14. Running SQL queries


In this step, let\'s execute SQL queries to retrieve specific records from the temporary view which was created earlier. Let\'s demonstrate how to display all records from the data table and filter those records based on vaccination totals. This capability allows for efficient data exploration and analysis using SQL syntax.


## Displaying All Records


The first query retrieves all records from the temporary view data using the SQL command SELECT * FROM data_v. The show() method is called to display the results in a tabular format. This is useful for getting an overview of the entire dataset.


In [19]:
spark.sql('SELECT * FROM data_v').show()

+-------------+-----------+------------+------------------+----------+
|    continent|total_cases|total_deaths|total_vaccinations|population|
+-------------+-----------+------------+------------------+----------+
|         Asia|     235214|        7998|                 0|  41128772|
|          nan|   13145380|      259117|                 0|1426736614|
|       Europe|     335047|        3605|                 0|   2842318|
|       Africa|     272139|        6881|                 0|  44903228|
|      Oceania|       8359|          34|                 0|     44295|
|       Europe|      48015|         159|                 0|     79843|
|       Africa|     107481|        1937|                 0|  35588996|
|North America|       3904|          12|                 0|     15877|
|North America|       9106|         146|                 0|     93772|
|South America|   10101218|      130663|                 0|  45510324|
|         Asia|     452273|        8777|                 0|   2780472|
|North

### Filtering records


The second query is designed to filter the data set to show only those continents where the total vaccinations exceed 1 million. The SQL command used here is `SELECT continent FROM data_v WHERE total_vaccinations > 1000000`. The `show()` method is again used to display the results, specifically listing the continents that meet the filter criteria.


In [ ]:
sv WHERE total_vaccinations > 1000000").show()

Displaying continent with total vaccinated more than 1 million:
+-------------+
|    continent|
+-------------+
|          nan|
|North America|
|       Europe|
|       Europe|
|          nan|
|          nan|
|          nan|
|         Asia|
|         Asia|
|       Europe|
|          nan|
|         Asia|
|      Oceania|
|          nan|
|          nan|
|          nan|
|          nan|
+-------------+



## Conclusion


This hands-on-lab is designed to help you understand the core functionalities of robust tools for data manipulation and analysis. By mastering these tools, you gain the ability to choose the right one for your data processing needs, whether working with large data sets requiring distributed computing or smaller data sets that lend themselves to quick manipulation in memory.


##  Author


**Ritika Joshi**


<!--## Change Log
|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2024-09-23|0.1|Ritika|First Draft|
--!>
